# GP Contract Services

Data on GP Contract services ( http://www.content.digital.nhs.uk/catalogue/PUB22004 ):

- Learning disabilities health check scheme (LDHC)
- Seasonal influenza vaccination programme (SFLU)
- Childhood seasonal influenza vaccination programme (CFLU)
- Facilitating timely diagnosis for people with dementia scheme (DEM)
- Named accountable GP for all patients (NAGP)

In [1]:
import pandas as pd
import sqlite3

In [2]:
ALL=4
MOST_RECENT=-1

In [3]:
#SET THIS
SHEETS=ALL

In [4]:
con = sqlite3.connect("gpcontracts.sqlite")

In [5]:
#http://www.content.digital.nhs.uk/catalogue/PUB22004

In [6]:
def downloader(typ,url=None):
    ''' Download and unzip data file '''
    !rm downloads/{typ}.zip
    if url is None:
        url='http://www.content.digital.nhs.uk/catalogue/PUB22004/{typ}.zip'.format(typ=typ)
    #Download the data from the HSCIC website
    !wget -P downloads/ {url}
    !rm -r data/{typ}/
    #Unzip the downloaded files into a subdirectory of the data folder, making sure the data dir exists first
    !mkdir -p data
    #The -o flag is overkill - if we hadn't deleted the original folder it would overwrite any similar files
    !unzip -o -d data/{typ} downloads/{typ}.zip
    
def downloader_xl(typ,url=None):
    if url is None:
        url='http://www.content.digital.nhs.uk/catalogue/PUB22004/{typ}_XLS.xlsx'.format(typ=typ)
        !rm data/{typ}_XLS.xlsx
    #Download the data from the HSCIC website
    !wget -P data/ {url}

In [7]:
def metadata_loader(con,typ,sheet='Note and definitions'):
    con.execute('DROP TABLE IF EXISTS metadata_{typ}'.format(typ=typ))
    df=pd.read_excel('data/{}_XLS.xlsx'.format(typ),sheet,skiprows=10,header=None)
    df.dropna(how='all',inplace=True)
    df.dropna(axis=1,how='all',inplace=True)
    df=df[:] if len(df) % 2 == 0 else df[:-1]
    tmp=pd.DataFrame({'code':df.iloc[::2][0].tolist(), 'explanation':df.iloc[1::2][0].tolist()})
    tmp=tmp[tmp['code'].isin(pd.read_sql_query('SELECT * FROM {typ} LIMIT 1'.format(typ=typ),con).columns.tolist())]
    tmp=tmp.set_index(['code'])
    tmp.to_sql(con=con, name='metadata_{typ}'.format(typ=typ),if_exists='append')

In [8]:
def db_loader(con,typ,sheets,keys,add_sheet_index=None):
    con.execute('DROP TABLE IF EXISTS {}'.format(typ))
    dff=pd.DataFrame()
    for sheet in sheets:
        print(typ,sheet)
        df=pd.read_excel('data/{}_XLS.xlsx'.format(typ),sheet,skiprows=9,na_values=[':','X',None])
        if add_sheet_index is not None:
            df[add_sheet_index]=sheet.strip('"').strip("'")
        dff=pd.concat([dff,df])
    tmp=dff.set_index(keys)
    #If the table exists, replace it, under the assumption we are using a more recent version of the data
    tmp.to_sql(con=con, name=typ,if_exists='append')

## Learning disabilities health check scheme 2015-16

In [9]:
typ='LDHC_1516'

In [10]:
#downloader('LDHC_1516_CSV')
downloader_xl(typ)

--2016-10-21 00:03:31--  http://www.content.digital.nhs.uk/catalogue/PUB22004/LDHC_1516_XLS.xlsx
Resolving www.content.digital.nhs.uk... 194.189.27.28
Connecting to www.content.digital.nhs.uk|194.189.27.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3409292 (3.3M) [application/vnd.ms-excel.12]
Saving to: 'data/LDHC_1516_XLS.xlsx'

LDHC_1516_XLS.xlsx  100%[=====================>]   3.25M  2.74MB/s   in 1.2s   

2016-10-21 00:03:33 (2.74 MB/s) - 'data/LDHC_1516_XLS.xlsx' saved [3409292/3409292]



In [11]:
xl=pd.ExcelFile('data/{}_XLS.xlsx'.format(typ))
xl.sheet_names

['Title sheet',
 'Note and definitions',
 'Data Quality',
 'Practice Data Quality',
 '30 June 2015',
 '30 September 2015',
 '30 December 2015',
 '31 March 2016']

In [12]:
df=pd.read_excel('data/{}_XLS.xlsx'.format(typ),xl.sheet_names[SHEETS],skiprows=9,na_values=':')
df.head(3)

,ONS Code,CCG Code,CCG Name,GP practice Code,GP practice Name,Achievement Date,Data Quality Issues,LDHC001 (Maximum),LDHC001 (Input),LDHCMI001,...,LDHCMI005,LDHCMI006,LDHCMI007,LDHCMI008,LDHCMI009,LDHCMI0010,LDHCMI0011,LDHCMI0012,LDHCMI0013,LDHCMI0014
0,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82007,TOWNHEAD SURGERY,2015-06-30,NaN,35,2,2,...,37.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82020,CROSSHILLS GROUP PRACTICE,2015-06-30,See data quality table,32,X,X,...,32.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82053,DYNELEY HOUSE SURGERY,2015-06-30,NaN,NaN,2,2,...,21.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:
#load the database
db_loader(con,typ,xl.sheet_names[SHEETS:],['GP practice Code','Achievement Date'])

LDHC_1516 30 June 2015
LDHC_1516 30 September 2015
LDHC_1516 30 December 2015
LDHC_1516 31 March 2016


/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [14]:
gpcode='J84005'
tmp=pd.read_sql_query('SELECT * from {typ} WHERE "GP practice Code"="{gpcode}"'.format(typ=typ,gpcode=gpcode), con)
tmp.head(3)

,GP practice Code,Achievement Date,ONS Code,CCG Code,CCG Name,GP practice Name,Data Quality Issues,LDHC001 (Maximum),LDHC001 (Input),LDHCMI001,...,LDHCMI005,LDHCMI006,LDHCMI007,LDHCMI008,LDHCMI009,LDHCMI0010,LDHCMI0011,LDHCMI0012,LDHCMI0013,LDHCMI0014
0,J84005,2015-06-30 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,63.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,J84005,2015-09-30 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,See data quality table,NaN,2.0,2.0,...,58.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
2,J84005,2015-12-31 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,See data quality table,NaN,1.0,1.0,...,58.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0


In [17]:
metadata_loader(con,typ,sheet=xl.sheet_names[1])
pd.read_sql_query('SELECT * from metadata_{typ}'.format(typ=typ), con)

,code,explanation
0,LDHC001 (Maximum),The number of patients aged 14 years or over i...
1,LDHC001 (Input),The number of those patients aged 14 years or ...
2,LDHCMI001,Quarterly count of registered patients aged 18...
3,LDHCMI002,Quarterly count of registered patients aged 14...
4,LDHCMI003,Quarterly count of registered patients aged 18...
5,LDHCMI004,Quarterly count of registered patients aged 14...
6,LDHCMI005,Quarterly (cumulative) count of registered pat...
7,LDHCMI006,Quarterly (cumulative) count of registered pat...
8,LDHCMI007,Quarterly count of registered patients aged 18...
9,LDHCMI008,Quarterly count of registered patients aged 14...


## Seasonal influenza vaccination programme 2015-16.csv

In [18]:
typ='SFLU_1516'

In [19]:
#downloader('SFLU_1516_CSV')
downloader_xl(typ)

--2016-10-21 00:06:20--  http://www.content.digital.nhs.uk/catalogue/PUB22004/SFLU_1516_XLS.xlsx
Resolving www.content.digital.nhs.uk... 194.189.27.28
Connecting to www.content.digital.nhs.uk|194.189.27.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6876759 (6.6M) [application/vnd.ms-excel.12]
Saving to: 'data/SFLU_1516_XLS.xlsx'

SFLU_1516_XLS.xlsx  100%[=====================>]   6.56M  2.82MB/s   in 2.3s   

2016-10-21 00:06:25 (2.82 MB/s) - 'data/SFLU_1516_XLS.xlsx' saved [6876759/6876759]



In [20]:
xl=pd.ExcelFile('data/{}_XLS.xlsx'.format(typ))
xl.sheet_names

['Title sheet',
 'Notes and definitions',
 'Data quality',
 'Practice data quality',
 'September 2015',
 'October 2015',
 'November 2015',
 'December 2015',
 'January 2016',
 'February 2016',
 'March 2016']

In [21]:
df=pd.read_excel('data/{}_XLS.xlsx'.format(typ),xl.sheet_names[SHEETS],skiprows=9,na_values=':')
df.head(3)

,ONS Code,CCG Code,CCG Name,Practice Code,Practice Name,Achievement Date,Data Quality,Registered patients (65+),SFLU001,SFLUMI002,...,SFLUMI011,Registered patients (6m to 9yrs),SFLU004,SFLU005,SFLUMI001,Registered patients (18+),SFLUMI012,SFLUMI013,Registered patients (18 to 64yrs),SFLUMI014
0,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82007,TOWNHEAD SURGERY,2015-09-30,NaN,2687.0,247.0,0.0,...,0.0,384.0,0,0,0,7879.0,0.0,0.0,5192.0,0.0
1,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82020,CROSSHILLS GROUP PRACTICE,2015-09-30,NaN,2705.0,534.0,0.0,...,0.0,686.0,0,0,0,9734.0,0.0,0.0,7029.0,0.0
2,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82028,FISHER MEDICAL CENTRE,2015-09-30,NaN,3556.0,440.0,NaN,...,NaN,756.0,0,0,NaN,11618.0,NaN,NaN,8062.0,NaN


In [22]:
#load the database
db_loader(con,typ,xl.sheet_names[SHEETS:],['Practice Code','Achievement Date'])

SFLU_1516 September 2015
SFLU_1516 October 2015
SFLU_1516 November 2015
SFLU_1516 December 2015
SFLU_1516 January 2016
SFLU_1516 February 2016
SFLU_1516 March 2016


/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [23]:
gpcode='J84005'
tmp=pd.read_sql_query('SELECT * from {typ} where "Practice Code"="{gpcode}"'.format(typ=typ,gpcode=gpcode), con)
tmp.head(3)

,Practice Code,Achievement Date,ONS Code,CCG Code,CCG Name,Practice Name,Data Quality,Registered patients (65+),SFLU001,SFLUMI002,...,SFLUMI011,Registered patients (6m to 9yrs),SFLU004,SFLU005,SFLUMI001,Registered patients (18+),SFLUMI012,SFLUMI013,Registered patients (18 to 64yrs),SFLUMI014
0,J84005,2015-10-31 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,2511.0,550.0,2.0,...,636.0,505.0,0,0,0,8343.0,188.0,31.0,5832.0,4.0
1,J84005,2015-11-30 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,2511.0,133.0,5.0,...,578.0,505.0,0,0,0,8343.0,192.0,35.0,5832.0,5.0
2,J84005,2015-12-31 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,2511.0,80.0,NaN,...,NaN,505.0,0,0,None,8343.0,None,None,5832.0,None


In [24]:
metadata_loader(con,typ,sheet=xl.sheet_names[1])
pd.read_sql_query('SELECT * from metadata_{typ}'.format(typ=typ), con)

,code,explanation
0,SFLU001,Monthly count of patients aged 65 years and ov...
1,SFLU002,Monthly count of the number of eligible patien...
2,SFLU003,Monthly count of the number of eligible patien...
3,SFLU004,Monthly count of the number of eligible patien...
4,SFLU005,Monthly count of the number of eligible patien...
5,SFLUMI001,Monthly count of eligible patients identified ...
6,SFLUMI002,Monthly count of patients aged 65 years and ov...
7,SFLUMI003,Monthly count of eligible patients identified ...
8,SFLUMI004,Monthly count of patients aged 65 years and ov...
9,SFLUMI005,Monthly count of patients identified as at ris...


## Childhood seasonal influenza vaccination programme 2015-16.csv


In [25]:
typ='CFLU_1516'

In [26]:
#downloader('CFLU_1516_CSV')
downloader_xl(typ)

--2016-10-21 00:08:45--  http://www.content.digital.nhs.uk/catalogue/PUB22004/CFLU_1516_XLS.xlsx
Resolving www.content.digital.nhs.uk... 194.189.27.28
Connecting to www.content.digital.nhs.uk|194.189.27.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4460849 (4.3M) [application/vnd.ms-excel.12]
Saving to: 'data/CFLU_1516_XLS.xlsx'

CFLU_1516_XLS.xlsx  100%[=====================>]   4.25M  2.69MB/s   in 1.6s   

2016-10-21 00:08:48 (2.69 MB/s) - 'data/CFLU_1516_XLS.xlsx' saved [4460849/4460849]



In [27]:
xl=pd.ExcelFile('data/{}_XLS.xlsx'.format(typ))
xl.sheet_names

['Title sheet',
 'Note and definitions',
 'Data Quality',
 'Data Quality Table',
 'September 2015',
 'October 2015',
 'November 2015',
 'December 2015',
 'January 2016',
 'February 2016',
 'March 2016']

In [28]:
df=pd.read_excel('data/{}_XLS.xlsx'.format(typ),xl.sheet_names[SHEETS],skiprows=9,na_values=':')
df.head(3)

,ONS Code,CCG Code,CCG Name,GP Practice Code,GP Practice Name,Data Quality,CFLU002,CFLU003,CFLUMI001,CFLUMI009,CFLUMI002,CFLUMI003,CFLUMI007,CFLUMI008,"Registered Patients (aged 2,3,4)"
0,E38000042,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,NaN,0,0,0,0,0,0.0,418.0,0.0,415.0
1,E38000042,00C,NHS DARLINGTON CCG,A83006,ORCHARD COURT SURGERY,NaN,0,0,0,0,0,0.0,274.0,0.0,270.0
2,E38000042,00C,NHS DARLINGTON CCG,A83010,MOORLANDS SURGERY,NaN,0,0,0,0,0,0.0,457.0,0.0,450.0


In [29]:
#load the database
db_loader(con,typ,xl.sheet_names[SHEETS:],['GP Practice Code','Period'],add_sheet_index='Period')

CFLU_1516 September 2015
CFLU_1516 October 2015
CFLU_1516 November 2015
CFLU_1516 December 2015
CFLU_1516 January 2016
CFLU_1516 February 2016
CFLU_1516 March 2016


/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [30]:
gpcode='J84005'
tmp=pd.read_sql_query('SELECT * from {typ} where "GP Practice Code"="{gpcode}"'.format(typ=typ,gpcode=gpcode), con)
tmp.head(3)

,GP Practice Code,Period,ONS Code,CCG Code,CCG Name,GP Practice Name,Data Quality,CFLU002,CFLU003,CFLUMI001,CFLUMI009,CFLUMI002,CFLUMI003,CFLUMI007,CFLUMI008,"Registered Patients (aged 2,3,4)"
0,J84005,December 2015,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,12.0,0.0,8.0,0.0,4.0,0.0,182.0,1.0,244.0
1,J84005,February 2016,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,3.0,0.0,0.0,0.0,3.0,0.0,183.0,0.0,244.0
2,J84005,January 2016,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,0.0,0.0,0.0,0.0,0.0,0.0,182.0,0.0,244.0


In [31]:
metadata_loader(con,typ,sheet=xl.sheet_names[1])
pd.read_sql_query('SELECT * from metadata_{typ}'.format(typ=typ), con)

,code,explanation
0,CFLU002,Monthly count of the number of patients aged t...
1,CFLU003,Monthly count of the number of patients aged t...
2,CFLUMI001,"The number of patients aged two, three and fou..."
3,CFLUMI009,"The number of patients aged two, three and fou..."
4,CFLUMI002,"The number of patients aged two, three and fou..."
5,CFLUMI003,"The number of patients aged two, three and fou..."
6,CFLUMI007,"The number of patients aged two, three and fou..."
7,CFLUMI008,"The number of patients aged two, three and fou..."


## Facilitating timely diagnosis for people with dementia scheme 2015-16.csv

In [32]:
typ='DEM_1516'

In [33]:
#downloader('DEM_1516_CSV')
downloader_xl(typ)

--2016-10-21 00:10:25--  http://www.content.digital.nhs.uk/catalogue/PUB22004/DEM_1516_XLS.xlsx
Resolving www.content.digital.nhs.uk... 194.189.27.28
Connecting to www.content.digital.nhs.uk|194.189.27.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1757027 (1.7M) [application/vnd.ms-excel.12]
Saving to: 'data/DEM_1516_XLS.xlsx'

DEM_1516_XLS.xlsx   100%[=====================>]   1.67M  2.69MB/s   in 0.6s   

2016-10-21 00:10:27 (2.69 MB/s) - 'data/DEM_1516_XLS.xlsx' saved [1757027/1757027]



In [34]:
xl=pd.ExcelFile('data/{}_XLS.xlsx'.format(typ))
xl.sheet_names

['Title sheet',
 'Note and definitions',
 'Data Quality',
 'Practice Data Quality',
 '"Assessment"',
 '"At risk"',
 '"Non professional carer"',
 '"Advanced care plan"']

In [35]:
df=pd.read_excel('data/{}_XLS.xlsx'.format(typ),xl.sheet_names[SHEETS],skiprows=9,na_values=':')
df.head(3)

,ONS Code,CCG Code,CCG Name,GP practice Code,GP practice Name,Data Quality Issues,DEMC201,DEMMI01,DEMMI06,DEMMI07,DEMMI08,DEMMI09
0,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82007,TOWNHEAD SURGERY,NaN,36,28.0,2,0,5,1
1,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82020,CROSSHILLS GROUP PRACTICE,NaN,226,4.0,200,0,17,2
2,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B82053,DYNELEY HOUSE SURGERY,NaN,47,2.0,11,0,2,3


In [36]:
#load the database
db_loader(con,typ,xl.sheet_names[SHEETS:],['GP practice Code','Type'],add_sheet_index="Type")

DEM_1516 "Assessment"
DEM_1516 "At risk"
DEM_1516 "Non professional carer"
DEM_1516 "Advanced care plan"


/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [37]:
gpcode='J84005'
tmp=pd.read_sql_query('SELECT * from {typ} WHERE "GP practice Code"="{gpcode}"'.format(typ=typ,gpcode=gpcode), con)
tmp.head(3)

,GP practice Code,Type,CCG Code,CCG Name,DEMC201,DEMMI01,DEMMI02,DEMMI03,DEMMI04,DEMMI05,...,DEMMI10,DEMMI11,DEMMI12,DEMMI13,DEMMI14,DEMMI15,DEMMI16,Data Quality Issues,GP practice Name,ONS Code
0,J84005,Advanced care plan,10L,NHS ISLE OF WIGHT CCG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,26.0,None,ESPLANADE SURGERY,E38000087
1,J84005,Assessment,10L,NHS ISLE OF WIGHT CCG,25.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ESPLANADE SURGERY,E38000087
2,J84005,At risk,10L,NHS ISLE OF WIGHT CCG,NaN,NaN,20.0,0.0,0.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,ESPLANADE SURGERY,E38000087


In [38]:
metadata_loader(con,typ,sheet=xl.sheet_names[1])
pd.read_sql_query('SELECT * from metadata_{typ}'.format(typ=typ), con)

,code,explanation
0,DEMC201,Number of patients with a record of receiving ...
1,DEMMI01,The number of patients with a record of declin...
2,DEMMI03,The number of patients recorded as being ‘at r...
3,DEMMI04,The number of patients recorded as being ‘at r...
4,DEMMI06,The number of patients with a record of receiv...
5,DEMMI08,The number of patients with a record of receiv...
6,DEMMI09,The number of patients with a record of receiv...
7,DEMMI10,The number of patients recorded as a non-profe...
8,DEMMI11,The number of patients recorded as a non-profe...
9,DEMMI12,The number of patients recorded as a non-profe...


## Named accountable GP for all patients 2015-16.csv

In [39]:
typ='NAGP_1516'

In [40]:
#downloader('NAGP_1516_CSV')
downloader_xl(typ)

--2016-10-21 00:10:54--  http://www.content.digital.nhs.uk/catalogue/PUB22004/NAGP_1516_XLS.xlsx
Resolving www.content.digital.nhs.uk... 194.189.27.28
Connecting to www.content.digital.nhs.uk|194.189.27.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7452505 (7.1M) [application/vnd.ms-excel.12]
Saving to: 'data/NAGP_1516_XLS.xlsx'

NAGP_1516_XLS.xlsx  100%[=====================>]   7.11M  2.56MB/s   in 2.8s   

2016-10-21 00:10:58 (2.56 MB/s) - 'data/NAGP_1516_XLS.xlsx' saved [7452505/7452505]



In [41]:
xl=pd.ExcelFile('data/{}_XLS.xlsx'.format(typ))
xl.sheet_names

['Title sheet',
 'Notes and definitions',
 'Data quality',
 'Practice data quality',
 'April 2015',
 'May 2015',
 'June 2015',
 'July 2015',
 'August 2015',
 'September 2015',
 'October 2015',
 'November 2015',
 'December 2015',
 'January 2016',
 'February 2016',
 'March 2016']

In [42]:
df=pd.read_excel('data/{}_XLS.xlsx'.format(typ),xl.sheet_names[SHEETS],skiprows=9,na_values=':')
df.head(3)

,ONS code,CCG code,CCG name,GP practice code,GP practice name,Achievement date,Data quality,NAGPMI011,NAGPMI001,NAGPMI002,NAGPMI003,NAGPMI004,NAGPMI005,NAGPMI006,NAGPMI007,NAGPMI008,NAGPMI009,NAGPMI010
0,E38000001,02N,"NHS AIREDALE, WHARFEDALE AND CRAVEN CCG",B83624,I G MEDICAL,2015-04-30,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,E38000002,09C,NHS ASHFORD CCG,G82735,SOUTH ASHFORD MEDICS,2015-04-30,See data quality table,8069.0,0.0,0.0,0.0,0.0,282.0,282.0,0.0,0.0,0.0,0.0
2,E38000004,07L,NHS BARKING AND DAGENHAM CCG,F82604,MARKS GATE HEALTH CENTRE,2015-04-30,NaN,3765.0,252.0,3513.0,0.0,0.0,219.0,219.0,0.0,0.0,0.0,0.0


In [43]:
#load the database
db_loader(con,typ,xl.sheet_names[SHEETS:],['GP practice code','Achievement date'])

NAGP_1516 April 2015
NAGP_1516 May 2015
NAGP_1516 June 2015
NAGP_1516 July 2015
NAGP_1516 August 2015
NAGP_1516 September 2015
NAGP_1516 October 2015
NAGP_1516 November 2015
NAGP_1516 December 2015
NAGP_1516 January 2016
NAGP_1516 February 2016
NAGP_1516 March 2016


/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [44]:
gpcode='J84005'
tmp=pd.read_sql_query('SELECT * from {typ} WHERE "GP practice code"="{gpcode}"'.format(typ=typ,gpcode=gpcode), con)
tmp.head(3)

,GP practice code,Achievement date,ONS code,CCG code,CCG name,GP practice name,Data quality,NAGPMI011,NAGPMI001,NAGPMI002,NAGPMI003,NAGPMI004,NAGPMI005,NAGPMI006,NAGPMI007,NAGPMI008,NAGPMI009,NAGPMI010
0,J84005,2015-09-30 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,9898.0,1333.0,8530.0,9.0,26.0,1146.0,1072.0,71.0,1.0,2.0,0.0
1,J84005,2015-10-31 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,9900.0,1451.0,8405.0,4.0,40.0,1135.0,1102.0,27.0,2.0,4.0,0.0
2,J84005,2015-11-30 00:00:00,E38000087,10L,NHS ISLE OF WIGHT CCG,ESPLANADE SURGERY,None,9884.0,3598.0,6240.0,8.0,38.0,1120.0,1095.0,20.0,4.0,1.0,0.0


In [45]:
metadata_loader(con,typ,sheet=xl.sheet_names[1])
pd.read_sql_query('SELECT * from metadata_{typ}'.format(typ=typ), con)

,code,explanation
0,NAGPMI001,The number of patients who have been allocated...
1,NAGPMI002,The number of patients who have been allocated...
2,NAGPMI003,The total number of patients who have no recor...
3,NAGPMI004,The number of patients who have not been alloc...
4,NAGPMI005,The number of patients on the GP practice list...
5,NAGPMI006,The number of patients aged 75 years or over w...
6,NAGPMI007,The number of patients aged 75 years or over w...
7,NAGPMI008,The total number of patients aged 75 years or ...
8,NAGPMI009,The number of patients aged 75 years or over w...
9,NAGPMI010,The number of patients aged 75 years or over w...
